# CppImport magic's documentation
`cppimport.magic` is an [IPython](http://ipython.org) extension that help to use C/C++ code in an interactive session.

* Author `cppimport`:  T. Ben Thompson (t.ben.thompson@gmail.com)
* `cppimport.magic`: Serguei E. Leontiev (leo@sai.msu.ru)
* Homepage: https://github.com/tbenthompson/cppimport
* SPDX-License-Identifier: MIT

TODO: Editorial edits are required. Sorry for my best Engish.

## Install or upgrade
You can install or upgrade via `pip`:
```
        pip install -U cppimport
```

## Usage
Then you are ready to load the magic:

In [1]:
%load_ext cppimport.magic

To load it each time IPython starts, list it in your configuration file:
```
    c.InteractiveShellApp.extensions = [
        'cppimport.magic'
    ]
```

In [2]:
%%cppimport --help #
#

::

  %cppimport [-v] [--help] cpp_module

Build and import C/C++ module from ``%%ccpimport`` cell

The content of the cell is written to a file in the
directory ``$IPYTHONDIR/cppimport/<random>/<hash>/`` using
a dirname with the hash of the code, flags and configuration
data. This file is then compiled. The resulting module is
imported.

Usage
=====
Prepend ``%%cppimport`` to your C++/C code in a cell:

%%cppimport module.cpp
// put your code here.

positional arguments:
  cpp_module       Module C/C++ source file name.

options:
  -v, --verbosity  Increase output verbosity.
  --help           Print docstring as output cell.



In [3]:
%cppimport_config --help

::

  %cppimport_config [-v] [--clean-cache] [--defaults] [--help]

options:
  -v, --verbosity  Increase output verbosity.
  --clean-cache    Clean ``cppimport.magic`` build cache.
  --defaults       Delete custom configuration and back to default.
  --help           Print docstring as output cell.



## Verbosity
By default, magic returns output data only if errors or warnings occur during compilation. But you can increase the verbosity with the flag ``-v``:
* ``-v`` - list top level objects of imporing module;
* ``-vv`` - setuptools verbose output;
* ``-vvv`` - debug logging.

In [4]:
%cppimport_config -v

INF:cppimport.magic:New default arguments for %%cppimport: -v


In [5]:
%cppimport_config --clean-cache
first_replay_test = len(In)

INF:cppimport.magic:Clean cache: /Users/leo/.cache/ipython/cppimport/def85c47


## C/C++ code cell's
You can use any interface libraries: [py11bind](https://pybind11.readthedocs.io), [Boost.Python](https://www.boost.org/doc/libs/release/libs/python/), native [Python/C API](https://docs.python.org/3/c-api/index.html) etc.

Additional build flags configure by `cfg` dictionary in [Mako](https://www.makotemplates.org/) code blocks (`<%` and `%>`).

### Use py11bind
`setup_pybind11(cfg)` - short configuration definition for the comparatively popular C++11/Python interface.

Also, classes defined by the cell (module) should be annotated as local (`py::module_local`). Classes without such an annotation will conflict with themselves when recompiled. See example [Use header files](#use_header_files) below.

In [6]:
%%cppimport somecode.cpp
// Source:
// https://github.com/tbenthompson/cppimport
#include <pybind11/pybind11.h>

namespace py = pybind11;

int square(int x) {
    return x * x + 1;
}

PYBIND11_MODULE(somecode, m) {
    m.def("square", &square);
}
/*<%
setup_pybind11(cfg)
%>*/

INF:cppimport.magic:C/C++ objects: somecode.square


In [28]:
somecode.square(12)

144

In [7]:
assert 4 == somecode.square(2)

#### C++14 py11bind example
To use [py11bind](https://pybind11.readthedocs.io) with C++14/17/23/... code, you must explicitly set the appropriate compiler flag.

In [8]:
%%cppimport cpp14module.cpp
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/cpp14module.cpp
#include <pybind11/pybind11.h>

namespace py = pybind11;

// Use auto instead of int to check C++14
auto add(int i, int j) {
    return i + j;
}

PYBIND11_MODULE(cpp14module, m) {
    m.def("add", &add);
}
/*<%
import pybind11
cfg['compiler_args'] = ['-std=c++14']
cfg['include_dirs'] = [pybind11.get_include(), pybind11.get_include(True)]
%>*/

INF:cppimport.magic:C/C++ objects: cpp14module.add


In [9]:
assert 18 == cpp14module.add(7, 11)

### Use C/Python API

In [10]:
%%cppimport raw_extension.c
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/raw_extension.c
#include <Python.h>

#if PY_MAJOR_VERSION >= 3
    #define MOD_INIT(name) PyMODINIT_FUNC PyInit_##name(void)
    #define MOD_DEF(ob, name, doc, methods) \
        static struct PyModuleDef moduledef = { \
            PyModuleDef_HEAD_INIT, name, doc, -1, methods, }; \
        ob = PyModule_Create(&moduledef);
    #define MOD_SUCCESS_VAL(val) val
#else
    #define MOD_INIT(name) PyMODINIT_FUNC init##name(void)
    #define MOD_DEF(ob, name, doc, methods) \
        ob = Py_InitModule3(name, methods, doc);
    #define MOD_SUCCESS_VAL(val)
#endif

static PyObject* add(PyObject* self, PyObject* args) {
    int a, b;
    //int class = 1;
    if (!PyArg_ParseTuple(args, "ii", &a, &b)) {
        return NULL;
    }
    return Py_BuildValue("i", a + b);
}

static PyMethodDef methods[] = {
    {"add", add, METH_VARARGS,
     "add(a: np.int32, b: np.int32) -> np.int32)"},
    {NULL}
};

MOD_INIT(raw_extension) {
    PyObject* m;
    MOD_DEF(m, "raw_extension", "", methods)
    return MOD_SUCCESS_VAL(m);
}

INF:cppimport.magic:C/C++ objects: raw_extension.add


In [11]:
assert 7 == raw_extension.add(2, 5)

### Use Boost.Python
To use [Boost.Python](https://www.boost.org/doc/libs/release/libs/python/), you must explicitly define libraries.

In [12]:
%%cppimport operators.cpp
// Source:
// https://github.com/TNG/boost-python-examples/blob/main/07-Operators/operators.cpp
#include <sstream>
#include <string>

class NumberLike {
public:
    NumberLike(int n = 0) : mN(n) {}
    NumberLike& operator+= (int i) {
        mN += i;
        return *this;
    }
    std::string str() const {
        std::stringstream s;
        s << mN;
        return s.str();
    }
    std::string repr() const {
        std::stringstream s;
        s << "NumberLike("<< mN << ")";
        return s.str();
    }
private:
    int mN;
};

NumberLike operator+(NumberLike n, int i) {    n += i;
    return n;
};

#include <boost/python.hpp>
using namespace boost::python;

BOOST_PYTHON_MODULE(operators) {
    class_<NumberLike>("NumberLike")
        .def(init< optional<int> >())
        .def(self + int())
        .def("__str__", &NumberLike::str)
        .def("__repr__", &NumberLike::repr)
    ;
}
/*<%
import os
import sys
_pfx = os.environ.get('CONDA_PREFIX', '/usr')
cfg['compiler_args'] += ['-std=c++14']
cfg['include_dirs'] += [ os.path.join(_pfx, 'include') ]
cfg['extra_link_args'] += ['-rpath', os.path.join(_pfx, 'lib')]
cfg['library_dirs'] += [ os.path.join(_pfx, 'lib') ]
cfg['libraries'] += ['boost_python' +
                     str(sys.version_info.major) + str(sys.version_info.minor)]
%>*/

INF:cppimport.magic:C/C++ objects: operators.NumberLike


In [27]:
n = operators.NumberLike(7)
m = n + 2
assert str(m) == '9'
n0 = operators.NumberLike()
m0 = n0 + 1
assert repr(m0) == 'NumberLike(1)'
print(n, m, str(m), n0, m0, repr(m0))

7 9 9 0 1 NumberLike(1)


## Use extra source
Since the module is built in a temporary directory, references to additional source files must be absolute.

In [14]:
%%writefile .temp_test_extra_sources1.cpp
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources1.cpp
int square(int x) {
    return x * x;
}

Overwriting .temp_test_extra_sources1.cpp


In [15]:
%%cppimport extra_sources.cpp
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources.cpp
#include <pybind11/pybind11.h>

int square(int x);

int square_sum(int x, int y) {
    return square(x) + square(y);
}

PYBIND11_MODULE(extra_sources, m) {
    m.def("square_sum", &square_sum);
}
/*<%
import os
setup_pybind11(cfg)
cfg['sources'] = [os.path.abspath('.temp_test_extra_sources1.cpp')]
cfg['parallel'] = True
%>*/

INF:cppimport.magic:C/C++ objects: extra_sources.square_sum


In [16]:
assert 25 == extra_sources.square_sum(3, 4)

<a id='use_header_files'></a>
## Use header files

Since the module is being built in a temporary directory, references to additional include directories, including current directory (`.`), must be absolute.

In [17]:
%%writefile .temp_test_thing.h
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/test_cppimport.py
#include <iostream>
struct Thing {
    void cheer() {
        std::cout << "WAHHOOOO" << std::endl;
    }
};
#define THING_DEFINED

Overwriting .temp_test_thing.h


In [18]:
%%writefile .temp_test_thing2.h
// This file is intentionally left empty.

Overwriting .temp_test_thing2.h


In [19]:
%%cppimport mymodule.cpp
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/mymodule.cpp
#include <pybind11/pybind11.h>
#include ".temp_test_thing.h"
#include ".temp_test_thing2.h"

namespace py = pybind11;

int add(int i, int j) {
    return i + j;
}

PYBIND11_MODULE(mymodule, m) {
    m.def("add", &add);
#ifdef THING_DEFINED
    // #pragma message "stuff"
    py::class_<Thing>(m, "Thing", py::module_local())
        .def(py::init<>())
        .def("cheer", &Thing::cheer);
#endif
}
/*<%
import os
setup_pybind11(cfg)
cfg['include_dirs'] += [os.path.abspath('.')]
cfg['dependencies'] += [os.path.abspath('.temp_test_thing.h')]
%>*/

INF:cppimport.magic:C/C++ objects: mymodule.add mymodule.Thing


In [20]:
assert 8 == mymodule.add(3, 5)

In [21]:
mymodule.Thing().cheer()

WAHHOOOO


## Internal `cppimport.magic`
Build and reloading module of the modified cell occurs as follows:
1. Calculate checksum


In [22]:
import sys

(mymodule.__file__,
 [k for k in sys.modules if 'mymodule' in k],
 mymodule.__name__)

('/Users/leo/.cache/ipython/cppimport/9a9a5adc/7015077fb6d486875b027ba04750bc2a/_7015077fb6d486875b027ba04750bc2a_mymodule.cpython-311-darwin.so',
 ['_7015077fb6d486875b027ba04750bc2a_mymodule'],
 'mymodule')

In [23]:
last_replay_test = len(In)

In [24]:
import re
import sys

import cppimport

force_assert_fail = -1 # 4
for test_case in [("cppimport.settings['force_rebuild'] = False;"
                   "get_ipython().run_line_magic('cppimport_config', '-v');"
                  ),
                  ("cppimport.settings['force_rebuild'] = True;"
                  ),
                  #("get_ipython().run_line_magic('cppimport_config', '-vvv');"
                  #)
                 ]:
    print('\033[95m exec( ',
      test_case,
      ' )\033[0m', flush=True)
    exec(test_case)
    for i in range(first_replay_test, last_replay_test-1):
        ii = In[i]
        if force_assert_fail != 0:
            force_assert_fail -= 1
        elif 'assert' in ii:
            ii = re.sub(r'(assert)\s\s*([0-9])', r'\1 1\2', ii)
            ii = re.sub(r'(assert)\s\s*([^0-9])', r'\1 " " + \2', ii)
        print('\033[92m exec(\033[0m',
              ii.replace('\\n', '\n').replace("\\'", "'"),
              '\033[92m )\033[0m', flush=True)
        exec(ii)
        sys.stdout.flush()
        sys.stderr.flush()

 exec(  cppimport.settings['force_rebuild'] = False;get_ipython().run_line_magic('cppimport_config', '-v');  )
INF:cppimport.magic:New default arguments for %%cppimport: -v
 exec( get_ipython().run_cell_magic('cppimport', 'somecode.cpp', '// Source:
// https://github.com/tbenthompson/cppimport
#include <pybind11/pybind11.h>

namespace py = pybind11;

int square(int x) {
    return x * x;
}

PYBIND11_MODULE(somecode, m) {
    m.def("square", &square);
}
/*<%
setup_pybind11(cfg)
%>*/
')  )


INF:cppimport.magic:C/C++ objects: somecode.square
 exec( assert 4 == somecode.square(2)  )
 exec( get_ipython().run_cell_magic('cppimport', 'cpp14module.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/cpp14module.cpp
#include <pybind11/pybind11.h>

namespace py = pybind11;

// Use auto instead of int to check C++14
auto add(int i, int j) {
    return i + j;
}

PYBIND11_MODULE(cpp14module, m) {
    m.def("add", &add);
}
/*<%
import pybind11
cfg['compiler_args'] = ['-std=c++14']
cfg['include_dirs'] = [pybind11.get_include(), pybind11.get_include(True)]
%>*/
')  )


INF:cppimport.magic:C/C++ objects: cpp14module.add
 exec( assert 18 == cpp14module.add(7, 11)  )
 exec( get_ipython().run_cell_magic('cppimport', 'raw_extension.c', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/raw_extension.c
#include <Python.h>

#if PY_MAJOR_VERSION >= 3
    #define MOD_INIT(name) PyMODINIT_FUNC PyInit_##name(void)
    #define MOD_DEF(ob, name, doc, methods) \\
        static struct PyModuleDef moduledef = { \\
            PyModuleDef_HEAD_INIT, name, doc, -1, methods, }; \\
        ob = PyModule_Create(&moduledef);
    #define MOD_SUCCESS_VAL(val) val
#else
    #define MOD_INIT(name) PyMODINIT_FUNC init##name(void)
    #define MOD_DEF(ob, name, doc, methods) \\
        ob = Py_InitModule3(name, methods, doc);
    #define MOD_SUCCESS_VAL(val)
#endif

static PyObject* add(PyObject* self, PyObject* args) {
    int a, b;
    //int class = 1;
    if (!PyArg_ParseTuple(args, "ii", &a, &b)) {
        return NULL;
    }
    return Py_BuildValue("i

INF:cppimport.magic:C/C++ objects: raw_extension.add
 exec( assert 7 == raw_extension.add(2, 5)  )
 exec( get_ipython().run_cell_magic('cppimport', 'operators.cpp', '// Source:
// https://github.com/TNG/boost-python-examples/blob/main/07-Operators/operators.cpp
#include <sstream>
#include <string>

class NumberLike {
public:
    NumberLike(int n = 0) : mN(n) {}
    NumberLike& operator+= (int i) {
        mN += i;
        return *this;
    }
    std::string str() const {
        std::stringstream s;
        s << mN;
        return s.str();
    }
    std::string repr() const {
        std::stringstream s;
        s << "NumberLike("<< mN << ")";
        return s.str();
    }
private:
    int mN;
};

NumberLike operator+(NumberLike n, int i) {    n += i;
    return n;
};

#include <boost/python.hpp>
using namespace boost::python;

BOOST_PYTHON_MODULE(operators) {
    class_<NumberLike>("NumberLike")
        .def(init< optional<int> >())
        .def(self + int())
        .def("__str__", &

INF:cppimport.magic:C/C++ objects: operators.NumberLike
 exec( n = operators.NumberLike(7)
m = n + 2
assert str(m) == '9'
n0 = operators.NumberLike()
m0 = n0 + 1
assert repr(m0) == 'NumberLike(1)'
print(n, m, str(m), n0, m0, repr(m0))  )
7 9 9 0 1 NumberLike(1)
 exec( get_ipython().run_cell_magic('writefile', '.temp_test_extra_sources1.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources1.cpp
int square(int x) {
    return x * x;
}
')  )
Overwriting .temp_test_extra_sources1.cpp
 exec( get_ipython().run_cell_magic('cppimport', 'extra_sources.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources.cpp
#include <pybind11/pybind11.h>

int square(int x);

int square_sum(int x, int y) {
    return square(x) + square(y);
}

PYBIND11_MODULE(extra_sources, m) {
    m.def("square_sum", &square_sum);
}
/*<%
import os
setup_pybind11(cfg)
cfg['sources'] = [os.path.abspath('.temp_test_extra_sources1.cpp')]
cfg['parallel'] 

INF:cppimport.magic:C/C++ objects: extra_sources.square_sum
 exec( assert 25 == extra_sources.square_sum(3, 4)  )
 exec( get_ipython().run_cell_magic('writefile', '.temp_test_thing.h', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/test_cppimport.py
#include <iostream>
struct Thing {
    void cheer() {
        std::cout << "WAHHOOOO" << std::endl;
    }
};
#define THING_DEFINED
')  )
Overwriting .temp_test_thing.h
 exec( get_ipython().run_cell_magic('writefile', '.temp_test_thing2.h', '// This file is intentionally left empty.
')  )
Overwriting .temp_test_thing2.h
 exec( get_ipython().run_cell_magic('cppimport', 'mymodule.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/mymodule.cpp
#include <pybind11/pybind11.h>
#include ".temp_test_thing.h"
#include ".temp_test_thing2.h"

namespace py = pybind11;

int add(int i, int j) {
    return i + j;
}

PYBIND11_MODULE(mymodule, m) {
    m.def("add", &add);
#ifdef THING_DEFINED
    // #prag

INF:cppimport.magic:C/C++ objects: mymodule.add mymodule.Thing
 exec( assert 8 == mymodule.add(3, 5)  )
 exec( mymodule.Thing().cheer()  )
WAHHOOOO
 exec( import sys

(mymodule.__file__,
 [k for k in sys.modules if 'mymodule' in k],
 mymodule.__name__)  )
 exec(  cppimport.settings['force_rebuild'] = True;  )
 exec( get_ipython().run_cell_magic('cppimport', 'somecode.cpp', '// Source:
// https://github.com/tbenthompson/cppimport
#include <pybind11/pybind11.h>

namespace py = pybind11;

int square(int x) {
    return x * x;
}

PYBIND11_MODULE(somecode, m) {
    m.def("square", &square);
}
/*<%
setup_pybind11(cfg)
%>*/
')  )
INF:cppimport.magic:C/C++ objects: somecode.square
 exec( assert 4 == somecode.square(2)  )
 exec( get_ipython().run_cell_magic('cppimport', 'cpp14module.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/cpp14module.cpp
#include <pybind11/pybind11.h>

namespace py = pybind11;

// Use auto instead of int to check C++14
auto add(int i, int 

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for NumberLike already registered; second conversion method ignored.


 exec( n = operators.NumberLike(7)
m = n + 2
assert str(m) == '9'
n0 = operators.NumberLike()
m0 = n0 + 1
assert repr(m0) == 'NumberLike(1)'
print(n, m, str(m), n0, m0, repr(m0))  )
7 9 9 0 1 NumberLike(1)
 exec( get_ipython().run_cell_magic('writefile', '.temp_test_extra_sources1.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources1.cpp
int square(int x) {
    return x * x;
}
')  )
Overwriting .temp_test_extra_sources1.cpp
 exec( get_ipython().run_cell_magic('cppimport', 'extra_sources.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources.cpp
#include <pybind11/pybind11.h>

int square(int x);

int square_sum(int x, int y) {
    return square(x) + square(y);
}

PYBIND11_MODULE(extra_sources, m) {
    m.def("square_sum", &square_sum);
}
/*<%
import os
setup_pybind11(cfg)
cfg['sources'] = [os.path.abspath('.temp_test_extra_sources1.cpp')]
cfg['parallel'] = True
%>*/
')  )
INF:cppimport.magic:C/C++ objects: ext

In [25]:
print('Successful magic doc test, 2+2 =', 2+2)

Successful magic doc test, 2+2 = 4
